# Race Trend Analysis
The goal of this notebook is to analyze trends within the entire data range and see what insights can be glean from the trends. 

In [101]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [102]:
df = pd.read_csv("datasets/UM _RACEDATA_RAW.csv") 

/var/folders/r6/hkw27m2d5416xj4s3mzmxqym0000gn/T/ipykernel_90524/798432201.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/UM _RACEDATA_RAW.csv")


Calculating ```athlete_age```

In [103]:
df['athlete_age'] = 2022 - df['Athlete year of birth']

Only ```USA``` runners

In [104]:
df = df[df['Athlete country'].isin(['USA'])]

Removing 
* ```Athlete club```
* ```Athlete country```
* ```Athlete year of birth```
* ```Athlete age category```
* ```Event number of finishers```

In [105]:
df = df.drop(['Athlete club', 'Athlete country', 'Athlete year of birth', 'Athlete age category', 'Event number of finishers'], axis = 1)

Rename column headers

In [106]:
df = df.rename(columns = { 'Year of event' : 'year',
                           'Event dates' : 'race_day',
                           'Event name' : 'race_name',
                           'Event distance/length' : 'race_length',
                           'Athlete performance' : 'athlete_performance',
                           'Athlete gender' : 'athlete_gender',
                           'Athlete average speed' : 'athlete_average_speed',
                           'Athlete ID' : 'athlete_id'
})

Removing ```null``` values

In [107]:
df = df.dropna()

Drop ```duplicate``` rows

In [108]:
df = df.drop_duplicates()

fix types
* ```athlete_age``` to ```int```
* ```athlete_average_speed``` to ```float```

In [109]:
df['athlete_age'] = df['athlete_age'].astype(int)

Convert the ```'athlete_average_speed'``` column to numeric, coercing errors to ```NaN```

In [110]:
df['athlete_average_speed'] = pd.to_numeric(df['athlete_average_speed'], errors='coerce')

In [111]:
df = df.dropna()

In [112]:
df['athlete_average_speed'] = df['athlete_average_speed'].astype(float)

Determine how many different race lengths there are and then filter by the 4 most popular ultramarathon race distances

In [113]:
print(df['race_length'].value_counts(dropna=False))

race_length
50km             620836
50mi             278362
100mi            117543
100km             57324
24h               46990
                  ...  
1006km                1
875km                 1
1016km                1
80km/3Etappen         1
72.5km                1
Name: count, Length: 971, dtype: int64


### NOTE: Break off and look at 24 hour races

Focus only on the most popular races: 
* ```50km```
* ```50mi```
* ```100km```
* ```100mi```


In [114]:
df = df[df['race_length'].isin(['50km', '50mi', '100mi', '100km'])]

Convert ```'race_length'``` into a ```category``` type

In [115]:
df['race_length'] = df['race_length'].astype('category')

Filter for ```Male``` & ```Female``` athletes

In [116]:
df = df[df['athlete_gender'].isin(['M', 'F'])]

Convert ```'athlete_gender'``` into a ```category``` type

In [117]:
df['athlete_gender'] = df['athlete_gender'].astype('category')

Remove ```'h'``` from the ```athlete_performace``` values

In [118]:
df['athlete_performance'] = df['athlete_performance'].str.split(' ').str.get(0)

Filter down to only ```USA``` races

In [119]:
df = df[df['race_name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA']

Remove ```(USA)``` from ```race_name```

In [120]:
df['race_name'] = df['race_name'].str.replace(r'\(USA\)', '', regex=True).str.strip()

Convert ```race_day``` to ```type: datetime```

In [121]:
# Function to standardize and validate dates
def standardize_date(date_str):
    # Handle date ranges by taking the first part
    if '-' in date_str:
        date_str = date_str.split('-')[0].strip()
    
    # Check if the date_str is too short to be valid
    if len(date_str) < 8:  # Shorter than "DD.MM.YYYY"
        return None  # Mark as invalid
    
    # Split date to check individual components
    parts = date_str.split('.')
    
    # Check for zero day or month
    if parts[0] == '00' or parts[1] == '00':
        return None  # Mark as invalid
    
    # Remove any trailing dots and ensure format is consistent
    return date_str.rstrip('.')

# Apply the standardization function
df['race_day'] = df['race_day'].apply(standardize_date)

# Remove rows with invalid dates
df = df.dropna(subset=['race_day'])

# Convert 'race_day' to datetime format, specifying day first
df['race_day'] = pd.to_datetime(df['race_day'], format='%d.%m.%Y', dayfirst=True)

Convert ```athlete_performace``` to ```type: timedelta[ns]```

In [122]:
df['athlete_performance'] = pd.to_timedelta(df['athlete_performance'])

Convert ```race_name``` to ```type: category```

In [123]:
df['race_name'] = df['race_name'].astype('category')

In [127]:
df['race_length'].value_counts()

race_length
50km     598105
50mi     247396
100km     36933
100mi      1023
Name: count, dtype: int64